In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = pd.read_csv('Train_modified.csv',encoding = "ISO-8859-1")
target = 'Disbursed'
IDcol = 'ID'

/home/cadu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12,14,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:

def loaddata(Filename):
    data = pd.read_csv(Filename,sep=',',header = None)
    return np.array(data)
# dataset
data1 = loaddata("data0_1.csv")
data2 = loaddata("data1_1.csv")
data3 = loaddata("data2_1.csv")
data4 = loaddata("data3_1.csv")
data5 = loaddata("data4_1.csv")

data_train = np.vstack((data1[:len(data1)-1],data2[:len(data1)]))
data_train = np.vstack((data_train,data3[:len(data1)]))
data_train = np.vstack((data_train,data4[:len(data1)]))
data_train = np.vstack((data_train,data5[:len(data1)]))

print('This is data_train',type(data_train),data_train.shape)
#label
data1 = loaddata("label0_1.csv")
data2 = loaddata("label1_1.csv")
data3 = loaddata("label2_1.csv")
data4 = loaddata("label3_1.csv")
data5 = loaddata("label4_1.csv")

label_train = np.vstack((data1[:len(data1)-1],data2[:len(data1)]))
label_train = np.vstack((label_train,data3[:len(data1)]))
label_train = np.vstack((label_train,data4[:len(data1)]))
label_train = np.vstack((label_train,data5[:len(data1)]))
#print(label_test[100:800])
train_X,test_X,train_Y,test_Y=cross_validation.train_test_split(data_train,label_train,test_size=0.1)

print('This is test_X',type(test_X),test_X.shape)
print('This is test_Y',type(test_Y),test_Y.shape)
print('This cell has done...')
train = train_X
predictors = test_X

This is data_train <class 'numpy.ndarray'> (25954, 661)
This is test_X <class 'numpy.ndarray'> (2596, 661)
This is test_Y <class 'numpy.ndarray'> (2596, 1)
This cell has done...


In [21]:

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_X, label=train_Y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds) #show_progress=False
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [27]:
#Choose all predictors except target & IDcols
#predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 nthread=12,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

SyntaxError: keyword can't be an expression (<ipython-input-27-3bc61e48de36>, line 14)